In [1]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd

In [2]:
# def input_layer():
#     data = pd.read_csv('data.csv')
#     return data

Step 1️⃣ — Imports

In [3]:
import torch
import torch.nn as nn
import math

Step 2️⃣ — Positional Encoding

Transformers don’t have recurrence or convolution, so we need position info.

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x


Step 3️⃣ — Transformer Encoder Block

This block includes Multi-Head Attention, Feed Forward, LayerNorm, and Residuals.

In [5]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self, d_model, n_heads, dim_ff, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_ff, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        # Self-attention
        attn_output, _ = self.self_attn(src, src, src)
        src = src + self.dropout1(attn_output)
        src = self.norm1(src)

        # Feed Forward
        ff_output = self.linear2(self.dropout(torch.relu(self.linear1(src))))
        src = src + self.dropout2(ff_output)
        src = self.norm2(src)
        return src


Step 4️⃣ — Full Transformer Encoder

Now stack multiple encoder blocks + positional encoding.

In [6]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model=512, n_heads=8, num_layers=6, dim_ff=2048, dropout=0.1, max_len=5000):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([
            TransformerEncoderBlock(d_model, n_heads, dim_ff, dropout) for _ in range(num_layers)
        ])
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        src = src.transpose(0, 1)  # (seq_len, batch, d_model)
        for layer in self.layers:
            src = layer(src)
        return src


Step 5️⃣ — Example Usage

In [7]:
# Example setup
vocab_size = 10000
seq_len = 20
batch_size = 4

model = TransformerEncoder(vocab_size)
sample_input = torch.randint(0, vocab_size, (batch_size, seq_len))  # (batch, seq_len)

output = model(sample_input)
print(output.shape)  # (seq_len, batch, d_model)


torch.Size([20, 4, 512])
